<a href="https://colab.research.google.com/github/NekoDK/Omgu_DA/blob/OmguDA_5/OmguDA_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_excel("zad5.xlsx")

In [ ]:
# Вычисление скользящего среднего с окном 4 и минимальным количеством периодов 3 (задавалось преподавателем)
df['av2'] = df['y'].rolling(window=4, min_periods=3).mean()
df['av2'] = df['av2'].shift(-1)
df.at[0, 'av2'] = None
df['av3'] = df['av2'].rolling(window=2).mean()
df['-y'] = df['y'] - df['av3']

In [ ]:
# Выделение сезонных компонент (задавалось преподавателем)
sf1 = df['-y'].iloc[4::4]
sf2 = df['-y'].iloc[5::4]
sf3 = df['-y'].iloc[6::4]
sf4 = df['-y'].iloc[3::4]

s1 = sf1.mean()
s2 = sf2.mean()
s3 = sf3.mean()
s4 = sf4.mean()

serr = (s1 + s2 + s3 + s4) / 4
s1 = s1 - serr
s2 = s2 - serr
s3 = s3 - serr
s4 = s4 - serr
s = [s1, s2, s3, s4]

In [ ]:
df['season'] = [s[i % 4] for i in range(len(df))]

In [ ]:
df['T+E = Y-S'] = df['y'] - df['season']

In [ ]:
def linearReg(X, y):
    model = LinearRegression()
    model.fit(X, y)
    y_predict = model.predict(X)
    return y_predict

In [ ]:
# Вычисление тренда с помощью линейной регрессии
y = df['T+E = Y-S'].values
df['Tcal'] = linearReg(df[['x']], y)

df['Ycal'] = df['Tcal'] + df['season']

df['Err^2'] = (df['Ycal'] - df['y'])**2

Ymean = df['y'].mean()
df['Y-Ymean'] = (df['y'] - Ymean)**2

In [ ]:
# Вычисление коэффициента детерминации (R²)
sumEr = df['Err^2'].sum()
sumMarkModel = df['Y-Ymean'].sum()
A = 1 - (sumEr / sumMarkModel)

In [ ]:
print(df)

     x      y         av2         av3         -y     season   T+E = Y-S  \
0    1  115.0         NaN         NaN        NaN -54.689063  169.689063   
1    2  236.4  205.633333         NaN        NaN  36.460938  199.939063   
2    3  265.5  174.275000  189.954167  75.545833  81.367188  184.132812   
3    4   80.2  164.700000  169.487500 -89.287500 -63.139062  143.339063   
4    5   76.7  144.375000  154.537500 -77.837500 -54.689063  131.389063   
5    6  155.1  128.275000  136.325000  18.775000  36.460938  118.639062   
6    7  201.1  121.600000  124.937500  76.162500  81.367188  119.732812   
7    8   53.5  115.625000  118.612500 -65.112500 -63.139062  116.639062   
8    9   52.8  109.300000  112.462500 -59.662500 -54.689063  107.489063   
9   10  129.8   98.250000  103.775000  26.025000  36.460938   93.339063   
10  11  156.9   98.650000   98.450000  58.450000  81.367188   75.532813   
11  12   55.1         NaN         NaN        NaN -63.139062  118.239062   

          Tcal        Yc

In [ ]:
print("Коэффициент детерминации (R²):", A)

Коэффициент детерминации (R²): 0.9388854972680405
